In [ ]:
!pip install tf-models-official
!pip install transformers
!pip install official
!pip install tensorflow
!pip install sentencepiece
!pip install emoji
!pip install vncorenlp
!pip install sentence_transformers

     |████████████████████████████████| 1.8 MB 8.6 MB/s 
     |████████████████████████████████| 4.9 MB 74.0 MB/s 
     |████████████████████████████████| 596 kB 62.0 MB/s 
     |████████████████████████████████| 213 kB 70.1 MB/s 
     |████████████████████████████████| 99 kB 13.4 MB/s 
     |████████████████████████████████| 90 kB 12.5 MB/s 
     |████████████████████████████████| 352 kB 78.4 MB/s 
     |████████████████████████████████| 1.2 MB 55.7 MB/s 
     |████████████████████████████████| 1.1 MB 35.6 MB/s 
     |████████████████████████████████| 47.6 MB 104 kB/s 
     |████████████████████████████████| 43 kB 2.4 MB/s 
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22258 sha256=1541f0236649bd27c88de9f068be54f682b6ff031651d2f83eb114ab5a40bc06
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=51273592422ce97

In [ ]:
import torch
import emoji
import re
import gc
import os
import json
import csv
import logging as lg
import pandas as pd
import numpy as np
import math
import tensorflow as tf
import official.nlp.optimization
from official import nlp

from tqdm import tqdm
from tensorflow.keras.utils import to_categorical
from sentence_transformers import SentenceTransformer
from tensorflow.keras.layers import *
from transformers import *
from tensorflow.keras.models import Model
from sklearn.model_selection import KFold, train_test_split
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score
from vncorenlp import VnCoreNLP
from nltk.tokenize import TweetTokenizer
from pandas import DataFrame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/public_train.csv"
val_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/val.csv"
test_path = "/content/drive/MyDrive/VLSP-Fake-News-Detection/final_private_test.csv"

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def normalizeToken(token):
    if len(token) == 1:
        return emoji.demojize(token)
    else:
        if token == "’":
            return "'"
        elif token == "…":
            return "..."
        else:
            return token

In [ ]:
def isnan(s):
    return s != s


def normalizePost(post, tweet_tokenizer, vncorenlp, use_segment=False, remove_punc_stopword=False, lowercase_opt=False, truncation_method="head_only", length=512):
    post = post.strip()
    URL_pattern = r"(?:http?s://|www.)[^\"]+"
    hashtag_pattern = r"#\w+"

    post = re.sub(URL_pattern, "", post)
    post = re.sub(hashtag_pattern, "hashtag", post)
    post = re.sub("\.+",".", post)
    #post = re.sub("#\s+", " ", post)
    post = re.sub("\*+", " ", post)
    post = re.sub("\$+", "đô ", post)
    post = re.sub("-{2,}", "", post)
    post = re.sub("\@+", "", post)
    post = re.sub("\[[0-9]?[0-9]]", " : dẫn_chứng ", post)

    post = post.strip()
    if lowercase_opt:
      post = post.lower()
    tokens = tweet_tokenizer.tokenize(post.replace("’", "'").replace("…", "..."))
    
    post = " ".join(tokens)
    if use_segment:
        tokens = vncorenlp.tokenize(post.replace("’", "'").replace("…", "..."))
        tokens = [t for ts in tokens for t in ts]
    normPost = " ".join(tokens)

    if remove_punc_stopword:
      tokens = [t for t in normPost if not t in vnmese_stopwords]
    normPost = " ".join(tokens)

    normPost = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normPost)
    normPost = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normPost)
    normPost = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normPost)
    if use_segment:
        normPost = normPost.replace('< url >', '<url>')
        normPost = re.sub(r"# (\w+)", r'#\1', normPost)
    if truncation_method == "head_only":
      normPost = " ".join(normPost.split(" ")[:length])
    if truncation_method == "tail_only":
      normPost = " ".join(normPost.split(" ")[-length:])
    if truncation_method == "head_tail":
      normPost = " ".join(normPost.split(" ")[:int(length*0.25)]) + " " +  " ".join(normPost.split(" ")[-int(length*0.75):])
    return normPost

In [ ]:
data_train = pd.read_csv(train_path)
data_val = pd.read_csv(val_path)
data_test = pd.read_csv(test_path)

In [ ]:
#Time stamp feature
data_train['timestamp_post'] = pd.to_numeric(data_train['timestamp_post'], errors='coerce')
data_train['timestamp_post'] = data_train['timestamp_post'].astype('float')

data_test['timestamp_post'] = pd.to_numeric(data_test['timestamp_post'], errors='coerce')
data_test['timestamp_post'] = data_test['timestamp_post'].astype('float')

data_val['timestamp_post'] = pd.to_numeric(data_val['timestamp_post'], errors='coerce')
data_val['timestamp_post'] = data_val['timestamp_post'].astype('float')

#Number of shares feature
data_train['num_share_post'] = pd.to_numeric(data_train['num_share_post'], errors='coerce')
data_train['num_share_post'] = data_train['num_share_post'].astype('float')

data_test['num_share_post'] = pd.to_numeric(data_test['num_share_post'], errors='coerce')
data_test['num_share_post'] = data_test['num_share_post'].astype('float')

data_val['num_share_post'] = pd.to_numeric(data_val['num_share_post'], errors='coerce')
data_val['num_share_post'] = data_val['num_share_post'].astype('float')

#Number of likes feature
data_train['num_like_post'] = pd.to_numeric(data_train['num_like_post'], errors='coerce')
data_train['num_like_post'] = data_train['num_like_post'].astype('float')

data_test['num_like_post'] = pd.to_numeric(data_test['num_like_post'], errors='coerce')
data_test['num_like_post'] = data_test['num_like_post'].astype('float')

data_val['num_like_post'] = pd.to_numeric(data_val['num_like_post'], errors='coerce')
data_val['num_like_post'] = data_val['num_like_post'].astype('float')

#Number of comments feature
data_train['num_comment_post'] = pd.to_numeric(data_train['num_comment_post'], errors='coerce')
data_train['num_comment_post'] = data_train['num_comment_post'].astype('float')

data_test['num_comment_post'] = pd.to_numeric(data_test['num_comment_post'], errors='coerce')
data_test['num_comment_post'] = data_test['num_comment_post'].astype('float')

data_val['num_comment_post'] = pd.to_numeric(data_val['num_comment_post'], errors='coerce')
data_val['num_comment_post'] = data_val['num_comment_post'].astype('float')

data_train['post_message'] = data_train['post_message'].fillna('empty')
data_val['post_message'] = data_val['post_message'].fillna('empty')

In [ ]:
def null_index_list(df, feature):
  return df[feature][df[feature].isnull()].index.tolist()

def fill_missing_median(df, feature):
  df[feature] = df[feature].fillna(df[feature].median())

In [ ]:
nan_timestamp_idxs = null_index_list(data_train, "timestamp_post")
nan_like_idxs = null_index_list(data_train, "num_like_post")
nan_share_idxs = null_index_list(data_train, "num_share_post")
nan_comment_idxs = null_index_list(data_train, "num_comment_post")

val_nan_timestamp_idxs = null_index_list(data_val, "timestamp_post")
val_nan_like_idxs = null_index_list(data_val, "num_like_post")
val_nan_share_idxs = null_index_list(data_val, "num_share_post")
val_nan_comment_idxs = null_index_list(data_val, "num_comment_post")

test_nan_timestamp_idxs = null_index_list(data_test, "timestamp_post")
test_nan_like_idxs = null_index_list(data_test, "num_like_post")
test_nan_share_idxs = null_index_list(data_test, "num_share_post")
test_nan_comment_idxs = null_index_list(data_test, "num_comment_post")

In [ ]:
tweet_tokenizer = TweetTokenizer()
rdrsegmenter = VnCoreNLP("/content/drive/MyDrive/VLSP-Fake-News-Detection/vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg")

train_sents = []
test_sents = []
val_sents = []

# If PhoBERT: use_segment = True, otherwise False
# normalizePost(post, tweet_tokenizer, vncorenlp, use_segment=False, remove_punc_stopword=False, lowercase_opt=False, truncation_method="head_only", length=512)
for post in data_train.post_message:
  train_sents.append(normalizePost(post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_only", 512))

for post in data_val.post_message:
  val_sents.append(normalizePost(post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_only", 512))

for test_post in data_test.post_message:
  test_sents.append(normalizePost(test_post, tweet_tokenizer, rdrsegmenter, False, False, False, "head_only", 512))

In [ ]:
from sentence_transformers import SentenceTransformer
sent_model = SentenceTransformer('bert-base-nli-mean-tokens')

train_sen_embeddings = sent_model.encode(train_sents)
test_sen_embeddings = sent_model.encode(test_sents)
val_sen_embeddings = sent_model.encode(val_sents)

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
train_pairwise_similarities = cosine_similarity(train_sen_embeddings)
val_pairwise_similarities = cosine_similarity(val_sen_embeddings)
test_pairwise_similarities = cosine_similarity(test_sen_embeddings)

In [ ]:
#Fill missing value by copy their most similar post message's values to them
def fill_missing_similarity(df, pairwise_similarity, feature, features_list):
  for i in features_list:
    max_val, pos_i, pos_j = -1, -1, -1
    for j in range(len(pairwise_similarity[i])):
      if i == j or j in features_list:
        continue
      elif pairwise_similarity[i][j] > max_val:
        #max_val = pairwise_similarities[i][j]
        pos_i = i
        pos_j = j
    df[feature][pos_i] = df[feature][pos_j]

In [ ]:
fill_missing_similarity(data_train, train_pairwise_similarities, "timestamp_post", nan_timestamp_idxs)
fill_missing_similarity(data_val, val_pairwise_similarities, "timestamp_post", val_nan_timestamp_idxs)
fill_missing_similarity(data_test, test_pairwise_similarities, "timestamp_post", test_nan_timestamp_idxs)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [ ]:
fill_missing_median(data_train, "num_like_post")
fill_missing_median(data_train, "num_share_post")
fill_missing_median(data_train, "num_comment_post")

fill_missing_median(data_val, "num_like_post")
fill_missing_median(data_val, "num_share_post")
fill_missing_median(data_val, "num_comment_post")

fill_missing_median(data_test, "num_like_post")
fill_missing_median(data_test, "num_share_post")
fill_missing_median(data_test, "num_comment_post")

In [ ]:
def create_additional_features(df):
  df['year'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.year
  df['month'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.month
  df['day'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.day
  df['weekday'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.weekday
  df['hour'] = pd.to_datetime(df['timestamp_post'], unit='s').dt.hour

In [ ]:
create_additional_features(data_train)
create_additional_features(data_val)
create_additional_features(data_test)

In [ ]:
data_train = data_train.drop(columns=["timestamp_post"])
data_val = data_val.drop(columns=["timestamp_post"])
data_test = data_test.drop(columns=["timestamp_post"])

In [ ]:
train_datetime_statements = []
for (day, month, hour, weekday) in zip(data_train['day'], data_train['month'], data_train['hour'], data_train['weekday']):
  if weekday != 6:
    train_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    train_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  train_datetime_statements.append(train_pattern_post)

print(train_datetime_statements)

['Bài viết được đăng vào thứ 6, ngày 3 tháng 4 lúc 20 giờ', 'Bài viết được đăng vào thứ 6, ngày 8 tháng 5 lúc 11 giờ', 'Bài viết được đăng vào thứ 7, ngày 6 tháng 6 lúc 1 giờ', 'Bài viết được đăng vào thứ 7, ngày 13 tháng 6 lúc 4 giờ', 'Bài viết được đăng vào thứ 2, ngày 9 tháng 3 lúc 7 giờ', 'Bài viết được đăng vào thứ 3, ngày 26 tháng 5 lúc 4 giờ', 'Bài viết được đăng vào thứ 4, ngày 13 tháng 5 lúc 19 giờ', 'Bài viết được đăng vào thứ 5, ngày 28 tháng 5 lúc 1 giờ', 'Bài viết được đăng vào thứ 4, ngày 15 tháng 4 lúc 11 giờ', 'Bài viết được đăng vào thứ 4, ngày 20 tháng 5 lúc 7 giờ', 'Bài viết được đăng vào thứ 5, ngày 21 tháng 5 lúc 5 giờ', 'Bài viết được đăng vào thứ 3, ngày 31 tháng 3 lúc 11 giờ', 'Bài viết được đăng vào thứ 2, ngày 13 tháng 4 lúc 1 giờ', 'Bài viết được đăng vào thứ 5, ngày 20 tháng 2 lúc 11 giờ', 'Bài viết được đăng vào thứ 6, ngày 26 tháng 6 lúc 8 giờ', 'Bài viết được đăng vào thứ 3, ngày 28 tháng 4 lúc 5 giờ', 'Bài viết được đăng vào thứ 4, ngày 29 tháng 4 lúc 0 

In [ ]:
val_datetime_statements = []
for (day, month, hour, weekday) in zip(data_val['day'], data_val['month'], data_val['hour'], data_val['weekday']):
  if weekday != 6:
    val_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    val_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  val_datetime_statements.append(val_pattern_post)

In [ ]:
test_datetime_statements = []
for (day, month, hour, weekday) in zip(data_test['day'], data_test['month'], data_test['hour'], data_test['weekday']):
  if weekday != 6:
    test_pattern_post = f"Bài viết được đăng vào thứ {weekday + 2}, ngày {day} tháng {month} lúc {hour} giờ"
  else:
    test_pattern_post = f"Bài viết được đăng vào Chúa nhật, ngày {day} tháng {month} lúc {hour} giờ"
  test_datetime_statements.append(test_pattern_post)

print(train_datetime_statements)

['Bài viết được đăng vào thứ 6, ngày 3 tháng 4 lúc 20 giờ', 'Bài viết được đăng vào thứ 6, ngày 8 tháng 5 lúc 11 giờ', 'Bài viết được đăng vào thứ 7, ngày 6 tháng 6 lúc 1 giờ', 'Bài viết được đăng vào thứ 7, ngày 13 tháng 6 lúc 4 giờ', 'Bài viết được đăng vào thứ 2, ngày 9 tháng 3 lúc 7 giờ', 'Bài viết được đăng vào thứ 3, ngày 26 tháng 5 lúc 4 giờ', 'Bài viết được đăng vào thứ 4, ngày 13 tháng 5 lúc 19 giờ', 'Bài viết được đăng vào thứ 5, ngày 28 tháng 5 lúc 1 giờ', 'Bài viết được đăng vào thứ 4, ngày 15 tháng 4 lúc 11 giờ', 'Bài viết được đăng vào thứ 4, ngày 20 tháng 5 lúc 7 giờ', 'Bài viết được đăng vào thứ 5, ngày 21 tháng 5 lúc 5 giờ', 'Bài viết được đăng vào thứ 3, ngày 31 tháng 3 lúc 11 giờ', 'Bài viết được đăng vào thứ 2, ngày 13 tháng 4 lúc 1 giờ', 'Bài viết được đăng vào thứ 5, ngày 20 tháng 2 lúc 11 giờ', 'Bài viết được đăng vào thứ 6, ngày 26 tháng 6 lúc 8 giờ', 'Bài viết được đăng vào thứ 3, ngày 28 tháng 4 lúc 5 giờ', 'Bài viết được đăng vào thứ 4, ngày 29 tháng 4 lúc 0 

In [ ]:
train_num_statements = []
for (num_like, num_share, num_comment) in zip(data_train['num_like_post'], data_train['num_share_post'], data_train['num_comment_post']):
  train_num_post = f"Bài viết có {num_like} lượt like, {num_share} lượt chia sẻ và {num_comment} bình luận"
  train_num_statements.append(train_num_post)

In [ ]:
val_num_statements = []
for (num_like, num_share, num_comment) in zip(data_val['num_like_post'], data_val['num_share_post'], data_val['num_comment_post']):
  val_num_post = f"Bài viết có {num_like} lượt like, {num_share} lượt chia sẻ và {num_comment} bình luận"
  val_num_statements.append(val_num_post)

In [ ]:
test_num_statements = []
for (num_like, num_share, num_comment) in zip(data_test['num_like_post'], data_test['num_share_post'], data_test['num_comment_post']):
  # if num_like > 50 or num_share > 8 or num_comment > 12:
  #   test_num_post = "Bài viết có nhiều tương_tác"
  # else:
  #   test_num_post = "Bài viết ít tương_tác"
  test_num_post = f"Bài viết có {num_like} lượt like, {num_share} lượt chia sẻ và {num_comment} bình luận"
  test_num_statements.append(test_num_post)

In [ ]:
train_modified_texts = []
for (train_num_post, train_datetime_post, tr_sent) in zip(train_num_statements, train_datetime_statements, train_sents):
  train_text = train_num_post + ". " + train_datetime_post + ". " + tr_sent
  train_modified_texts.append(train_text)

In [ ]:
val_modified_texts = []
for (val_num_post, val_datetime_post, val_sent) in zip(val_num_statements, val_datetime_statements, val_sents):
  val_text = val_num_post + ". " + val_datetime_post  + ". " + val_sent
  val_modified_texts.append(val_text)

In [ ]:
test_modified_texts = []
for (test_num_post, test_datetime_post, test_sent) in zip(test_num_statements, test_datetime_statements, test_sents):
  test_text = test_num_post + ". " + test_datetime_post + ". " + test_sent
  test_modified_texts.append(test_text)

In [ ]:
MAX_LEN  = 256
MODEL = 'FPTAI/vibert-base-cased'
num_labels = 1

print("Loading BERT tokenizer")
tokenizer = BertTokenizer.from_pretrained(MODEL, do_lower_case=False)

Loading BERT tokenizer


Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

In [ ]:
train_input_ids = []
train_mask = []
for train_sent in tqdm(train_modified_texts):
  encoded_dict = tokenizer.encode_plus(train_sent, add_special_tokens=True, max_length=MAX_LEN,
                                       padding=True, return_attention_mask=True, return_tensors="np", truncation=True)
  train_input_ids.append(encoded_dict['input_ids'])
  train_mask.append(encoded_dict['attention_mask'])

100%|██████████| 4372/4372 [00:12<00:00, 350.81it/s]


In [ ]:
val_input_ids = []
val_mask = []
for val_sent in tqdm(val_modified_texts):
  encoded_dict = tokenizer.encode_plus(train_sent, add_special_tokens=True, max_length=MAX_LEN,
                                       padding=True, return_attention_mask=True, return_tensors="np", truncation=True)
  val_input_ids.append(encoded_dict['input_ids'])
  val_mask.append(encoded_dict['attention_mask'])

100%|██████████| 875/875 [00:01<00:00, 674.96it/s]


In [ ]:
BATCH_SIZE = 32
X_train = [train_input_ids, train_mask]
X_val = [val_input_ids, val_mask]

In [ ]:
def cnn_model(transformer_model, max_len=MAX_LEN):
  logits = []

  input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
  attention_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
  cls_output = transformer_model(input_ids, attention_mask)[0]

  dense = Dense(512)(extra_features)
  bn = BatchNormalization()(dense)
  dense = Activation("relu")(bn)
  logits.append(dense)

  convs = []
  kernel_sizes = [2, 3, 4, 5]
  size_pool = 5

  for size in kernel_sizes:
    l_conv = Conv1D(filters=256, kernel_size=size)(cls_output)
    l_conv = BatchNormalization()(l_conv)
    l_conv = Activation("relu")(l_conv)
    l_pool = MaxPooling1D(pool_size=size_pool)(l_conv)
    convs.append(l_pool)
  
  l2_pool = Concatenate(axis=1)(convs)
  for _ in range(3):
    origin  = l2_pool
    l2_conv = Conv1D(filters=256, kernel_size=size_pool,padding='same')(l2_pool)
    l2_pool = BatchNormalization()(l2_pool)
    l2_pool = Activation("relu")(l2_pool)
    l2_conv = Add()([origin, l2_conv])
    l2_pool = MaxPooling1D(pool_size=size_pool)(l2_conv)
  
  text = Flatten()